In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_project")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_ecommerce_sales = spark.table(f"{catalogo}.{esquema_source}.ecommerce_sales")

In [0]:
df_ecommerce_sales = df_ecommerce_sales.where(col("ecommerce_interaction_type") == "purchase")

In [0]:
df_ingresos_por_categoria = df_ecommerce_sales.groupBy(col("customer_category")).agg(
                                                     count(col("customer_id")).alias("ventas"),
                                                     sum(col("purchase_amount_USD")).alias("total_ingresos"),
                                                     round(avg(col("purchase_amount_USD")),2).alias("ticket_promedio")
                                                     ).where(col("customer_category") != "null").orderBy(col("customer_category").asc())


In [0]:
df_ingresos_por_genero = df_ecommerce_sales.groupBy(col("gender")).agg(
                                                     count(col("gender")).alias("Clientes"),
                                                     sum(col("purchase_amount_USD")).alias("total_ingresos"),
                                                     round(avg(col("purchase_amount_USD")),2).alias("ticket_promedio")
                                                     ).orderBy(col("gender").asc()).where(col("gender") != "null")

In [0]:
df_ingresos_por_suscripcion = df_ecommerce_sales.groupBy(col("subscription_status")).agg(
                                                     count(col("subscription_status")).alias("Clientes"),
                                                     sum(col("purchase_amount_USD")).alias("total_ingresos"),
                                                     round(avg(col("purchase_amount_USD")),2).alias("ticket_promedio")
                                                     ).orderBy(col("subscription_status").asc()).where(col("subscription_status") != "null")

In [0]:
df_ingresos_por_estado = df_ecommerce_sales.groupBy(col("location")).agg(
                                                     count(col("location")).alias("Clientes"),
                                                     sum(col("purchase_amount_USD")).alias("total_ingresos"),
                                                     round(avg(col("purchase_amount_USD")),2).alias("ticket_promedio")
                                                     ).orderBy(col("location").asc()).where(col("location") != "null")

In [0]:

df_ingresos_por_categoria.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.ingresos_por_categoria")
df_ingresos_por_genero.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.ingresos_por_genero")
df_ingresos_por_suscripcion.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.ingresos_por_suscripcion")
df_ingresos_por_estado.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.ingresos_por_estado")
